In [ ]:
using Revise
using ITensors
#using ITensorTDVP
import MSSTA

In [ ]:
R = 4

sitesx = [Index(2, "Qubit, x=$n") for n in 1:R]
sitesy = [Index(2, "Qubit, y=$n") for n in 1:R]
sitesz = [Index(1, "z=$n") for n in 1:R] # dummy index of size 1

In [ ]:
sitesxy = collect(Iterators.flatten(zip(sitesx, sitesy)))
sitesyz = collect(Iterators.flatten(zip(sitesy, sitesz)))

In [ ]:
A = randomMPS(sitesxy; linkdims=2)

In [ ]:
B = randomMPS(sitesy; linkdims=2)

In [ ]:
# Insert dummy z indices
B_ext = MPS([x for x in MSSTA.matchsiteinds(B, sitesyz)])

In [ ]:
AB_ext = MSSTA.automul(A, B_ext; tag_row="x", tag_col="z", tag_shared="y", alg="fit")

In [ ]:
# Remove the dummy index z
AB = MPS([AB_ext[2n-1] * AB_ext[2n] * onehot(sitesz[n]=>1) for n in 1:R])

In [ ]:
# Check
AB_reconst = vec(Array(reduce(*, AB), reverse(sitesx)))

Amat = reshape(Array(reduce(*, A), vcat(reverse(sitesx), reverse(sitesy))), 2^R, 2^R)
Bvec = vec(Array(reduce(*, B), reverse(sitesy)))
Amat * Bvec ≈ AB_reconst